<a href="https://colab.research.google.com/github/ChangED1122/Python/blob/Python-PyTorch/PyTorch_%E5%A4%A7%E9%87%8F%E6%95%B8%E6%93%9A%E5%BD%B1%E5%83%8F%E8%BE%A8%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests #發送網址請求

url = "https://firebasestorage.googleapis.com/v0/b/grandmacan-2dae4.appspot.com/o/ML_data%2Fone_piece_full.zip?alt=media&token=937656fd-f5c1-44f5-b174-1e2d590b8ef3"
with open("one_piece_full.zip","wb") as f:  #wb 二進制格式
  req = requests.get(url)
  f.write(req.content)

In [2]:
import zipfile

with zipfile.ZipFile("one_piece_full.zip","r") as zip_file:
  zip_file.extractall("one_piece_full") #解壓縮

In [3]:
with open("one_piece_full/classnames.txt","r") as f :
  lines = f.readlines()
  classes = [line.strip() for line in lines] #使用strip()將 \n 移除
  print(classes) #取得種類

['Ace', 'Akainu', 'Brook', 'Chopper', 'Crocodile', 'Franky', 'Jinbei', 'Kurohige', 'Law', 'Luffy', 'Mihawk', 'Nami', 'Rayleigh', 'Robin', 'Sanji', 'Shanks', 'Usopp', 'Zoro']


In [4]:
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image
from torchvision import transforms

class ImageDataset(Dataset):  #創建類別，繼承Dataset
  def __init__(self, root, train, transform=None): #初始化

    if train:
      image_root = Path(root) / "train"
    else :
      image_root = Path(root) / "test"

    with open(Path(root)/ "classnames.txt","r") as f :
      lines = f.readlines()
      self.classes = [line.strip( ) for line in lines] #classes=所有類別的列表，使用strip()將 \n 移除

    self.paths = [i for i in image_root.rglob("*") if i.is_file()]  #所有圖片路徑
    self.transform = transform
    
  
  def __getitem__(self, index):
    img = Image.open(self.paths[index]).convert("RGB")
    class_name = self.paths[index].parent.name #取得資料夾名稱（角色名）
    class_idx = self.classes.index(class_name) #取得角色之類別索引

    if self.transform:
      return self.transform(img), class_idx 
    else:
      return img, class_idx

    
  def __len__(self):
    return len(self.paths) #回傳圖片張數
    

In [5]:
train_transforms = transforms.Compose([ #把以上指令結合
    transforms.Resize((64,64)),
    transforms.TrivialAugmentWide(), # 隨機幫圖片做轉換 只要在訓練集做轉換就好
    transforms.ToTensor()
])

test_transforms = transforms.Compose([ 
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

In [6]:
train_dataset = ImageDataset(root = "one_piece_full",
                             train=True, 
                             transform = train_transforms)
test_dataset = ImageDataset(root = "one_piece_full",
                             train=False, 
                             transform = test_transforms)


In [7]:
len(train_dataset),len(test_dataset)

(3015, 750)

In [8]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=False)

In [9]:
len(train_dataloader), len(test_dataloader) #3016/16, 750/16

(189, 47)

In [10]:
from torch import nn
class ImageClassificationModel3(nn.Module):
  def __init__(self, input_shape, output_shape):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
      nn.Conv2d(in_channels=input_shape,
                out_channels=8, 
                kernel_size=(3,3), 
                stride=1, 
                padding=1
      ),
      nn.ReLU(),
      nn.Conv2d(in_channels=8,
                out_channels=8, 
                kernel_size=(3,3), 
                stride=1, 
                padding=1
      ),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2,2), 
                   stride=2, 
                   padding=0
      )
    )
    self.conv_block_2 = nn.Sequential(
      nn.Conv2d(in_channels=8,
                out_channels=16, 
                kernel_size=(3,3), 
                stride=1, 
                padding=1
      ),
      nn.ReLU(),
      nn.Conv2d(in_channels=16,
                out_channels=16, 
                kernel_size=(3,3), 
                stride=1, 
                padding=1
      ),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2,2), 
                   stride=2, 
                   padding=0
      )
    )
    self.classifier = nn.Sequential(
        nn.Flatten(start_dim=1,end_dim=-1), #預設是（1,-1）這樣沒有全部攤平
        nn.Linear(in_features=16*16*16, out_features=output_shape) #如果不知道in_features要填多少，可以先跑block_1&block_2
        #nn.Softmax()  因為torch的 CrossEntropy已經包含 Softmax()
    )
    

  def forward(self,x):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x


In [11]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
len(train_dataset.classes)

18

In [13]:
len(train_dataloader)

189

In [14]:
model_2 = ImageClassificationModel3(3, len(train_dataset.classes)) #input_shape彩色圖片 =3 , output_shape Y的類別
model_2.to(device)

ImageClassificationModel3(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4096, out_features=18, bias=True)
  )
)

In [15]:
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_2.parameters(), lr=0.001) 

In [16]:
def accuracy_fn(y_pred, y_true):
  correct_num = ( y_pred==y_true).sum()
  acc = correct_num/len(y_true) *100
  return acc

In [17]:
def train_step(dataloader, model, cost_fn, optimizer, accuracy_fn, device): #訓練步驟
   train_cost = 0
   train_acc = 0

   for batch, (x, y) in enumerate(dataloader):  # enumerate( ) 知道第幾個batch
    x= x.to(device)
    y= y.to(device)
    model.train()

    y_pred = model(x)

    cost = cost_fn(y_pred, y)
    train_cost += cost   #  train_cost的值=每個batch的cost加總

    train_acc += accuracy_fn(y_pred.argmax(dim=1) ,y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
  
   train_cost /= len(train_dataloader)  #取平均
   train_acc /= len(train_dataloader)
   print(f"\nTrain Cost: {train_cost:.4f}, Train Acc: {train_acc:.2f}")

def test_step(dataloader, model, cost_fn, accuracy_fn, device): #測試步驟
  test_cost = 0
  test_acc = 0
  model.eval()
  with torch.inference_mode():
    for x, y in dataloader:
      x= x.to(device)
      y= y.to(device)
      test_pred = model(x)
      
      test_cost += cost_fn(test_pred, y)
      test_acc += accuracy_fn(test_pred.argmax(dim=1) ,y)

    test_cost /= len(test_dataloader)
    test_acc /= len(test_dataloader)
  
  print(f"Test  Cost: {test_cost:.4f}, Test  Acc: {test_acc:.2f}  \n")

In [ ]:
from tqdm.auto import tqdm  #進度條

epochs = 10
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n------")
  train_step(train_dataloader, model_2, cost_fn, optimizer, accuracy_fn, device)

  test_step(test_dataloader, model_2, cost_fn, accuracy_fn, device)

遷移學習 transfer Learning

https://pytorch.org/vision/stable/models/generated/torchvision.models.efficientnet_b0.html#torchvision.models.EfficientNet_B0_Weights

In [ ]:
import torchvision
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT #取得參數
model = torchvision.models.efficientnet_b0(weights=weights)
model.to(device)

In [22]:
!pip install torchinfo
from torchinfo import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#將模型架構條列出來
summary(model = model,
        input_size=(16,3,64,64), # 跑完EfficientNet 變成 [16, 1000]
        col_names=["input_size", "output_size", "num_params", "trainable" ],
        row_settings=["var_names"])  #多了每一層的變數名稱

調整模型 因借來的模型數出值有1000個類別 ＝> 改成18個值

classifier底下需更改. # model.名稱[哪一層]

改完回去跑summary 檢查

In [59]:
from torch import nn
model.classifier[0] #Dropout層
model.classifier[1] = nn.Linear(in_features=1280, out_features=18, bias=True).to(device) # 更改Linear層中out_features=1000


除了最後一層（classifier）需要重新訓練以外，上面每層參數鎖住（沒有追蹤梯度） ，訓練的時候參數值就不會變

trainable -> False

In [ ]:
for param in  model.features.parameters():
  param.requires_grad=False
  print (param)

了解 EfficientNet 在訓練時須經過什麼樣的轉換

我們的資料集也需要相同轉換

In [61]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT #取得參數
EfficientNet_b0_transforms = weights.transforms()

In [62]:
train_dataset = ImageDataset(root="one_piece_full",
                             train=True, 
                             transform=EfficientNet_b0_transforms )
test_dataset = ImageDataset(root="one_piece_full",
                             train=False, 
                             transform=EfficientNet_b0_transforms )

In [63]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=False)

In [57]:
len(train_dataloader),len(test_dataloader)

(189, 47)

In [64]:
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
from tqdm.auto import tqdm  #進度條

epochs = 10
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n------")
  train_step(train_dataloader, model, cost_fn, optimizer, accuracy_fn, device)

  test_step(test_dataloader, model, cost_fn, accuracy_fn, device)

In [68]:
img = Image.open("1161514004.jpg").convert("RGB") #上面怎麼讀取跟轉換這裡一樣
img
img = EfficientNet_b0_transforms(img)
img.shape # [3,224,224]
img = img.reshape(-1,3,224,224) 


model.eval()
with torch.inference_mode():
  y_pred = model(img.to(device))

y_pred = torch.softmax(y_pred, dim=1) #轉換成機率
class_idx = y_pred.argmax(dim=1)
train_dataset.classes[class_idx]


'Zoro'